In [1]:
import numpy as np
import tensorflow as tf
from recommender import Recommender
import pandas as pd

In [3]:


test_list_ds = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/rating_lists_10/test").cache()
portfolios = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/portfolios_tfds").cache()

items_ids = portfolios.batch(10000).map(lambda x: x["STOCKCODE"])
unique_item_ids = np.unique(np.concatenate(list(items_ids)))

item_names = portfolios.batch(10000).map(lambda x: x["STOCKNAME"])
unique_item_names = np.unique(np.concatenate(list(item_names)))

item_GICS = portfolios.batch(10000).map(lambda x: x["GICS"])
unique_item_gics = np.unique(np.concatenate(list(item_GICS)))

user_ids = portfolios.batch(10000).map(lambda x: x["CDSACCNO"])
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

model = Recommender(
    loss = tf.keras.losses.MeanSquaredError(),
    portfolios = portfolios
    )

model.load_weights(r"D:\dev work\recommender systems\Atrad_CARS\model_weights\2024_04_29\tf_listwise_ranking_2024_04_29_16_42")
#2024_04_29\tf_listwise_ranking_2024_04_29_16_42 

In [4]:
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
model.evaluate(test_list_ds.shuffle(100_000).batch(1024), return_dict=True)

(None, 10, 32)  |  (None, 10, 64)
(None, 10, 32)  |  (None, 10, 64)
1/1 [==============================] - 3s 3s/step - ndcg_metric: 0.7140 - root_mean_squared_error: 0.9975 - loss: 0.9950 - regularization_loss: 0.0000e+00 - total_loss: 0.9950


{'ndcg_metric': 0.714043140411377,
 'root_mean_squared_error': 0.9975025057792664,
 'loss': 0.9950112104415894,
 'regularization_loss': 0,
 'total_loss': 0.9950112104415894}

In [5]:
symbol_uni_path = r"D:\dev work\recommender systems\Atrad_CARS\data\stock_data.xlsx"
stock_info = pd.read_excel(symbol_uni_path)
stock_info = stock_info.drop(['Unnamed: 0'],axis = 1)
stock_info.shape


(282, 4)

In [6]:
stock_info.head(3)

,symbol,name,buisnesssummary,gics_code
0,HBS,hSenid Business Solutions PLC,An indigenous multinational catering towards m...,45103010 - Application Software
1,TYRE,KELANI TYRES PLC,Manufacturing tyres and tubes and marketing lo...,Automobiles & Components
2,ABL,AMANA BANK PLC,unknown,Banks


In [7]:
len(stock_info['symbol'].to_list())


282

In [8]:
symbol_universe = np.array(stock_info['symbol'].to_list())
name_universe = np.array(stock_info['name'].to_list())
gics_universe = np.array(stock_info['gics_code'].to_list())


In [9]:
test_1 = next(iter(test_list_ds.batch(2)))
test_1

{'STOCKNAME': <tf.Tensor: shape=(2, 10), dtype=string, numpy=
 array([[b'KELANI CABLES PLC', b'BAIRAHA FARMS PLC',
         b'CEYLON COLD STORES PLC', b'R I L PROPERTY PLC',
         b'MERCHANT BANK OF SRI LANKA & FINANCE PLC', b'RENUKA FOODS PLC',
         b'AGALAWATTE  PLANTATIONS  PLC', b'HVA FOODS PLC',
         b'RENUKA HOTELS PLC', b'LANKA CREDIT AND BUSINESS FINANCE PLC'],
        [b'SARVODAYA DEVELOPMENT FINANCE PLC', b'MARAWILA  RESORTS  PLC',
         b'LANKEM DEVELOPMENTS PLC', b'HVA FOODS PLC',
         b'CITIZENS DEVELOPMENT BUSINESS FINANCE PLC',
         b'KELANI VALLEY PLANTATIONS PLC',
         b'CEYLON BEVERAGE HOLDINGS PLC', b'SIERRA CABLES PLC',
         b'DIALOG AXIATA PLC', b'HELA APPAREL HOLDINGS PLC']], dtype=object)>,
 'CDSACCNO': <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'RPS-20882-LI/00', b'RPS-890073360-VN/00'], dtype=object)>,
 'STOCKCODE': <tf.Tensor: shape=(2, 10), dtype=string, numpy=
 array([[b'KCAB', b'BFL', b'CCS', b'RIL', b'MBSL', b'COCO', 

In [10]:
test_1['CDSACCNO'].numpy()

array([b'RPS-20882-LI/00', b'RPS-890073360-VN/00'], dtype=object)

In [26]:
id2gics = dict(zip(stock_info.symbol, stock_info.gics_code))
id2name = dict(zip(stock_info.symbol, stock_info.name))

In [19]:
user = {
    'CDSACCNO' : test_1['CDSACCNO'].numpy(),
    'STOCKCODE' : symbol_universe[:20].reshape(-1,10),
    'GICS' : gics_universe[:20].reshape(-1,10),
    'STOCKNAME' : name_universe[:20].reshape(-1,10)
    }

In [20]:
symbol_universe[:20].reshape(-1,10)

array([['HBS', 'TYRE', 'ABL', 'DFCC', 'COMB', 'HNB', 'HDFC', 'NTB',
        'NDB', 'PABC'],
       ['SEYB', 'SDB', 'SAMP', 'UBC', 'HNBF', 'AEL', 'ACL', 'SPEN',
        'AFS', 'BRWN']], dtype='<U4')

In [21]:
user

{'CDSACCNO': array([b'RPS-20882-LI/00', b'RPS-890073360-VN/00'], dtype=object),
 'STOCKCODE': array([['HBS', 'TYRE', 'ABL', 'DFCC', 'COMB', 'HNB', 'HDFC', 'NTB',
         'NDB', 'PABC'],
        ['SEYB', 'SDB', 'SAMP', 'UBC', 'HNBF', 'AEL', 'ACL', 'SPEN',
         'AFS', 'BRWN']], dtype='<U4'),
 'GICS': array([['45103010 - Application Software', 'Automobiles & Components',
         'Banks', 'Banks', 'Banks', 'Banks', 'Banks', 'Banks', 'Banks',
         'Banks'],
        ['Banks', 'Banks', 'Banks', 'Banks', 'Banks Finance & Insurance',
         'Capital Goods', 'Capital Goods', 'Capital Goods',
         'Capital Goods', 'Capital Goods']], dtype='<U34'),
 'STOCKNAME': array([['hSenid Business Solutions PLC', 'KELANI TYRES PLC',
         'AMANA BANK PLC', 'DFCC BANK PLC',
         'COMMERCIAL BANK OF CEYLON PLC', 'HATTON NATIONAL BANK PLC',
         'HOUSING DEVELOPMENT FINANCE CORPORATION BANK OF SL',
         'NATIONS TRUST BANK PLC', 'NATIONAL DEVELOPMENT BANK PLC',
         'PAN ASIA 

In [22]:
model(user)

(2, 10, 32)  |  (2, 10, 64)


<tf.Tensor: shape=(2, 10, 1), dtype=float32, numpy=
array([[[1.5881217 ],
        [1.2460905 ],
        [2.2676735 ],
        [1.8818375 ],
        [1.6938076 ],
        [2.0285084 ],
        [2.2777996 ],
        [2.3224337 ],
        [2.7566538 ],
        [1.1188542 ]],

       [[0.97796226],
        [1.0023836 ],
        [1.2344999 ],
        [1.134217  ],
        [1.1520604 ],
        [0.9284433 ],
        [1.4246619 ],
        [1.0848207 ],
        [1.0204049 ],
        [1.2487178 ]]], dtype=float32)>